# Домашнее задание : ML как HTTP-сервис

## Задача 1: применяем PCA-трансформацию

Модифицируйте файл `train.py` - добавьте в пайплайн обучения модели сжатие размерности до `n_components=2` с помощью [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) и обучите модель **в докере** на "сжатых" данных. Сохраните полученный объект `pca_transformer.pkl`, который умеет выполнять сжатие данных.

Решением домашки считается модифицированный файл *train.py*

[Ссылка на файл train.py](docker_example/train.py)

In [6]:
import pickle
import os
import logging
from pathlib import Path

import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA

LOG_FORMAT = '%(asctime)s | %(levelname)-8s | %(filename)-25.25s:%(lineno)-4d | %(message)s'
#log_filename = "/www/classifier/data/service.log"  # для Docker
log_filename = "service.log"  # для Jupyter
logging.basicConfig(filename=log_filename, level=logging.INFO, format=LOG_FORMAT)

# загрузка данных
data_source = np.genfromtxt('data/client_segmentation.csv', delimiter=',', skip_header=1)
X = data_source[:, :3]
y = data_source[:, 3]

# сжатие размерности
pca = PCA(n_components=2)
X = pca.fit_transform(X)

# обучение модели
clf = DecisionTreeClassifier(max_depth=3, random_state=42)
clf.fit(X, y)

# сохраняем модель внутри контейнера в директории /www/classifier
with open('data/clf.pkl', 'wb') as f:
    pickle.dump(clf, f)
    logging.info('Модель обучена и сохранена в %s' % Path().absolute())
with open('data/pca_transformer.pkl', 'wb') as f:
    pickle.dump(pca, f)
    logging.info('Объект для сжатия данных сохранен в %s' % Path().absolute())
print(f"Модель обучена! Лог: {log_filename}")

Модель обучена! Лог: service.log


## Задача 2: трансформация входных фичей на лету

Модифицируйте файл `service.py`: добавьте загрузку объекта для трансформации `pca_tansformer.pkl` и применяйте её **в докере** для трансформации набора входных фич в сжатые (см. метод `.transform()`):
<pre>
[x1, x2, x3] -> [x1_pca, x2_pca]
</pre>

Соответственно, predict надо выполнять на *сжатых* фичах

Решением домашки считается модифицированный файл *service.py*

[Ссылка на файл service.py](docker_example/service.py)

In [17]:
import logging
import pickle
import numpy as np

# файл, куда посыпятся логи модели
LOG_FORMAT = '%(asctime)s | %(levelname)-8s | %(filename)-25.25s:%(lineno)-4d | %(message)s'
logging.basicConfig(filename="service.log", level=logging.INFO, format=LOG_FORMAT)


def parse_params(params) -> dict:
    """ Получаем и трансформируем параметры """
    
    params_list = params.split('&')
    params_dict = {'x1': None, 'x2': None, 'x3': None}
    for param in params_list:
        key, value = param.split('=')
        params_dict[key] = float(value)
    print(params_dict)
    params_full = np.array(list(params_dict.values()))  # все параметры
    params_pca = pca.transform(params_full.reshape(1, -1)).flatten()  # сжимаем до 2 параметров
    params_dict_pca = {'x1_pca': params_pca[0], 'x2_pca': params_pca[1]}  # создаем новый словарь
    return params_dict_pca


logging.info('Загружаем объект для трансформации')
with open('data/pca_transformer.pkl', 'rb') as f:
    pca = pickle.load(f)
    logging.info('Объект загружен: %s' % pca)
        
logging.info('Загружаем обученную модель')
with open('data/clf.pkl', 'rb') as f:
    classifier_model = pickle.load(f)
    logging.info('Модель загружена: %s' % classifier_model)

    
params_dict = parse_params('x1=1&x2=-2.2&x3=1.05')
print(params_dict)

response = params_dict
user_features = np.array([params_dict['x1_pca'], params_dict['x2_pca']]).reshape(1, -1)
predicted_class = int(classifier_model.predict(user_features)[0])
logging.info('predicted_class %s' % predicted_class)
print({'predicted_class': predicted_class})

{'x1': 1.0, 'x2': -2.2, 'x3': 1.05}
{'x1_pca': -0.10804973294238474, 'x2_pca': 0.6849826557247147}
{'predicted_class': 0}


## Задача 3: Используем Flask

Перепишите сервис на использование Flask. Вы можете взять готовый базовый образ с Flask, либо добавить установку в тот контейнер, который есть - это нужно сделать в Dockerfile

Задача необязательная, для успешного выполнения домашки достаточно первых двух пунктов

**Выполнение**. Вместо `service.py` был создан файл `app.py`, с использованием Flask для обработки запросов

[Ссылка на файл app.py](flask_app/app.py)

In [26]:
import webbrowser
import logging
import pickle
import numpy as np
import os
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
app = Flask(__name__)

def load_model():
    # файл, куда посыпятся логи модели
    LOG_FORMAT = '%(asctime)s | %(levelname)-8s | %(filename)-25.25s:%(lineno)-4d | %(message)s'

    logFormatter = logging.Formatter(LOG_FORMAT)
    rootLogger = logging.getLogger()

    fileHandler = logging.FileHandler("service.log")
    fileHandler.setFormatter(logFormatter)
    rootLogger.addHandler(fileHandler)

    consoleHandler = logging.StreamHandler()
    consoleHandler.setFormatter(logFormatter)
    rootLogger.addHandler(consoleHandler)

    logging.info('Загружаем объект для трансформации')
    with open('data/pca_transformer.pkl', 'rb') as f:
        pca = pickle.load(f)
        logging.info('Объект загружен: %s' % pca)

    logging.info('Загружаем обученную модель')
    with open('data/clf.pkl', 'rb') as f:
        classifier_model = pickle.load(f)
        logging.info('Модель загружена: %s' % classifier_model)
    return pca, classifier_model

def launch_classifier():
    param_x1 = (np.random.randint(100) - 50) / 10
    param_x2 = (np.random.randint(100) - 50) / 10
    param_x3 = (np.random.randint(100) - 50) / 10
    params_dict = {'x1': param_x1, 'x2': param_x2, 'x3': param_x3}

    pca, classifier_model = load_model()

    params_full = np.array(list(params_dict.values()))  # все параметры
    params_pca = pca.transform(params_full.reshape(1, -1)).flatten()  # сжимаем до 2 параметров
    params_dict_pca = {'x1_pca': params_pca[0], 'x2_pca': params_pca[1]}  # создаем новый словарь
    result = params_dict_pca

    user_features = np.array([params_dict_pca['x1_pca'], params_dict_pca['x2_pca']]).reshape(1, -1)
    predicted_class = int(classifier_model.predict(user_features)[0])
    result.update({'predicted_class': predicted_class})
    logging.info('predicted_class %s' % predicted_class)
    return params_dict, result


# @app.route('/ping')
def pong_response():
    return 'pong'


# @app.route('/hello')
def hello_world():
    return 'Hello, people. Every human is awesome'


# @app.route('/english')
def english():
    return webbrowser.open("https://www.youtube.com/watch?v=HbvYeLxMKN8")


# @app.route('/quokka')
def quokka_pictures():
    return webbrowser.open("https://www.instagram.com/explore/tags/quokka/")

In [24]:
pong_response()

'pong'

In [36]:
input_var, results = launch_classifier()
print('Classifier prediction')
print(f"Input variables: x1 = {input_var['x1']}, x2 = {input_var['x2']}, x3 = {input_var['x3']}")
print('-----------------------------------')
print(f"x1_pca = {results['x1_pca']}")
print(f"x2_pca = {results['x2_pca']}")
print(f"predicted_class = {results['predicted_class']}")

Classifier prediction
Input variables: x1 = 1.5, x2 = -3.9, x3 = 0.5
-----------------------------------
x1_pca = -1.051169986974994
x2_pca = 0.7724234362960463
predicted_class = 0


# Домашнее задание: рекомендательная система

## Задача 1: строим KNN

В реальной жизни KNN-рекомендатель не стоит делать на основе `sklearn.neighbors.NearestNeighbors` - есть готовые реализации, заточенные специально для построения рекомендательных систем. Хорошим примером такой реализации является [пакет implictit](). В рамках домашней работы предлагается разобраться с реализацией KNN-рекомендателя из этой библиотеки 

Почитайте документацию по модулю `implicit.nearest_neighbours.CosineRecommender`. Обучите KNN-рекомендатель и воспользуйтесь методом `recommend` для построения рекомендаций


In [1]:
import pandas as pd
import implicit
from implicit.nearest_neighbours import CosineRecommender
import numpy as np


content_views = pd.read_csv(
    'recsys_data/content_views.zip', delimiter=',', header=0, compression='zip',
    names = ['user_id', 'content_id', 'view_duration', 'view_ts', 'dt', 'platform'],
    dtype = {'user_id': np.uint32, 'content_id': np.uint16, 'view_duration': np.uint16},
    parse_dates = [3, 4]
)
print('Количество просмотров %s' % content_views.user_id.count())

content_views.head()

Количество просмотров 489565


,user_id,content_id,view_duration,view_ts,dt,platform
0,4649,52867,735,2019-03-18 20:40:57+03:00,2019-03-18,LG
1,16,48800,361,2019-03-18 11:48:27+03:00,2019-03-18,LG
2,5380,47146,268,2019-02-17 13:06:33+03:00,2019-02-17,LG
3,4498,30191,297,2019-03-18 15:27:18+03:00,2019-03-18,LG
4,4886,39349,302,2019-03-18 12:08:16+03:00,2019-03-18,LG


In [2]:
content_description = pd.read_csv(
    'recsys_data/content_description.zip', delimiter=',', header=0, compression='zip',
    names = ['content_id', 'origin_country', 'release_date', 'kinopoisk_rating', 'compilation_id', 'genre'],
    dtype = {'content_id': np.uint16},
    parse_dates = [2]
)
print('Количество доступного контента %s' % content_description.content_id.count())
content_description.head()

Количество доступного контента 126182


,content_id,origin_country,release_date,kinopoisk_rating,compilation_id,genre
0,1974,87.0,2009-12-15,7.27,153,Для детей
1,2148,87.0,2009-12-21,7.27,153,Для детей
2,2184,87.0,2009-12-22,7.27,153,Для детей
3,2443,1.0,2010-01-06,5.79,121,Романтические
4,2463,1.0,2010-01-06,6.80,515,Военные


In [3]:
from sklearn.preprocessing import LabelEncoder

# кодируем индексы пользователей
user_encoder = LabelEncoder()
user_encoder.fit(content_views.user_id)

# переиндексация контента
content_views = content_views.assign(
    user_index = user_encoder.transform(content_views.user_id)
)

# кодируем индексы контента
item_encoder = LabelEncoder()
item_encoder.fit(content_views.content_id)

# новая переиндексация
content_views = content_views.assign(
    item_index = item_encoder.transform(content_views.content_id)
)

content_views.head()

,user_id,content_id,view_duration,view_ts,dt,platform,user_index,item_index
0,4649,52867,735,2019-03-18 20:40:57+03:00,2019-03-18,LG,802,22812
1,16,48800,361,2019-03-18 11:48:27+03:00,2019-03-18,LG,2,20399
2,5380,47146,268,2019-02-17 13:06:33+03:00,2019-02-17,LG,911,19628
3,4498,30191,297,2019-03-18 15:27:18+03:00,2019-03-18,LG,773,13517
4,4886,39349,302,2019-03-18 12:08:16+03:00,2019-03-18,LG,836,16959


In [4]:
from scipy.sparse import csr_matrix

num_users = content_views.user_index.max() + 1
num_items = content_views.item_index.max() + 1
num_interactions = content_views.shape[0]

user_item = csr_matrix(
    (np.ones(num_interactions),(content_views.user_index.values, content_views.item_index.values)),
    shape=(num_users, num_items)
)
print('sparsity: %.4f' % (num_interactions / (num_users * num_items)))

user_item

sparsity: 0.0091


<2000x27012 sparse matrix of type '<class 'numpy.float64'>'
	with 259994 stored elements in Compressed Sparse Row format>

In [5]:
from sklearn.model_selection import train_test_split

train_ids, test_ids = train_test_split(
    np.arange(start=0, stop=user_item.shape[0], step=1, dtype=np.uint32),
    test_size=0.2
)
print(
    """
        Размер обучающей выборки %d пользователей
        Размер валидационной выборки %d пользователей
    """
    % (train_ids.size, test_ids.size)
)


        Размер обучающей выборки 1600 пользователей
        Размер валидационной выборки 400 пользователей
    


In [13]:
cos_model = implicit.nearest_neighbours.CosineRecommender()

# обучаемся только на тренировочной части пользователей
cos_model.fit(user_item[train_ids,:])

In [16]:
# пример рекомендаций для случайного пользователя
random_user_index = np.random.choice(test_ids)
random_user_history = user_item.getrow(random_user_index).reshape(1, -1)

recs = cos_model.recommend(random_user_index, user_item)
print('user_index %d, history: %s' % (random_user_index, random_user_history.nonzero()[1][:10]))
print('recommendations: %s' % recs)

user_index 1859, history: [921]
recommendations: [(99, 1.0), (600, 1.0), (224, 0.15422992461890894), (782, 0.05992215177119797), (1127, 0.05778319966569877), (511, 0.04296358767810755), (1032, 0.03507153119159472), (1504, 0.030513909884867668), (1280, 0.026171196129510688), (80, 0.02340181886004545)]


## Задача 2: Item to Item

Решите задачу c2c рекомендаций - вызовите метод `similar_items` для  *item_id=1*

In [9]:
related = cos_model.similar_items(itemid=1)
related

[(1, 1.0),
 (437, 0.0756830471617321),
 (296, 0.07146276141286449),
 (811, 0.06468462273531508),
 (242, 0.03593265345403678),
 (1441, 0.032826608214930636),
 (224, 0.030845984923781787),
 (1504, 0.030513909884867668),
 (568, 0.02918542027088895),
 (1536, 0.02878368312517019)]

## Задача 3: обучаем Implicit

Почитайте документацию по модулю implicit.als.AlternatingLeastSquares. Обучите ALS-рекомендатель и воспользуйтесь методом recommend для построения рекомендаций

In [10]:
als_model = implicit.als.AlternatingLeastSquares()
als_model.fit(user_item[train_ids,:])

In [12]:
# пример рекомендаций для случайного пользователя
random_user_index = np.random.choice(test_ids)
random_user_history = user_item.getrow(random_user_index).reshape(1, -1)

recs = als_model.recommend(random_user_index, user_item)
print('user_index %d, history: %s' % (random_user_index, random_user_history.nonzero()[1][:10]))
print('recommendations: %s' % recs)

user_index 114, history: [10390 10725 11615 18698 20928 21733 23009]
recommendations: [(614, 0.96021646), (1197, 0.7906255), (530, 0.6134136), (1593, 0.35570723), (1295, 0.30060688), (1430, 0.22426924), (980, 0.15876634), (1132, 0.15178873), (1173, 0.13222319), (1208, 0.12919477)]


## Задача 4: реализация метрик

Даны два вектора - истинная история пользователя и объекты, которые считает релеватными ваша модель

Вычислите

* precision
* recall
* precision@5


In [37]:
import numpy as np

user_interactions = [47315, 30004, 36322,  8942, 30820,  6086,  9126,   332, 16289,
       39106, 39335, 48506, 48654,  9234, 29935,  2678, 36202, 22636, 18007, 39328, 15414, 30016, 35601,
    58409, 21313,   386, 16303, 4397, 19644, 51887, 21659, 36325, 53030,  7764, 50266, 58734, 53419, 24121,
    50806, 36092,  8868, 28037, 36131, 13561, 16298, 27508, 41722, 30189, 46490,  2676, 43328, 781, 48397,
    41369, 39324, 36381, 39635, 27710, 47837, 28525, 12024, 56604, 41664, 37387, 48507, 413, 33526, 20059,
    49781, 56648, 16283, 50805, 34254, 39325, 59374, 22620,  8865, 27512, 13875, 30011,  7621,
    10544, 28076, 29716, 30054, 20490, 29466, 16852, 39363, 34250, 7024, 33541,   263, 21267, 25690, 23020,
    41368, 53414,  2681, 30201] 

user_recs = [
    50820, 27781, 36131, 50812, 36092, 12024, 59155, 30042, 15414, 19882, 21659, 27849, 39328, 34240, 2681,
    21267, 50126, 58560, 7764, 49781
]

cross = sum(elem in user_recs for elem in user_interactions)
precision = cross / len(user_recs)
recall = cross / len(user_interactions)
print('precision =', precision)
print('recall =', recall)

apak = 0
for index, element in enumerate(user_interactions):
    if element in user_recs:
        apak += 1 / (index + 1)
        
apak = apak / len(user_recs)
print('average precision at K =', apak)

precision = 0.5
recall = 0.1
average precision at K = 0.012958509750701858
